In [1]:
!pip install --upgrade \
    ag2[openai] \
    autogen-ext[mcp]==0.6.1 \
    gradio \
    tavily-python \
    httpx \
    sseclient-py \
    python-dotenv \
    uv \
    atlantis-open-weather-mcp \
    fastmcp \
    mcp[cli] \
    nest_asyncio \
    dash==2.10.2 \
    flask \
    dash-bootstrap-components \
    -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.4/306.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.3/185.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os

def cleanup_coding_folder():
    folder = "coding"
    if not os.path.exists(folder):
        print(f"Folder {folder} does not exist.")
        return

    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if os.path.isfile(filepath):
            os.remove(filepath)
            print(f"Deleted {filepath}")
        else:
            print(f"Skipped non-file: {filepath}")

# Call this before starting the pipeline
cleanup_coding_folder()


Skipped non-file: coding/.gradio
Deleted coding/donor_eligibility_design.md
Deleted coding/tmp_code_ae1e90f3455428971fad1af011616ae2.shell
Deleted coding/tmp_code_264bf4cfec8cb5399f1e66d4fb1ae1f7.shell
Skipped non-file: coding/__pycache__
Deleted coding/donor_eligibility.py
Deleted coding/app.py
Deleted coding/test_donor_eligibility.py


In [2]:
import os
import re
import subprocess
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import CodeBlock
from autogen.coding import LocalCommandLineCodeExecutor
import datetime
from typing import Optional, List, Dict, Union, Any
from uuid import uuid4
import gradio
import sys

# lm setup ----------------------------------
from google.colab import userdata
os.environ["ACCUWEATHER_API_KEY"] = userdata.get('ACCUWEATHER_API_KEY')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_BASE_URL"] = userdata.get('AZURE_OPENAI_BASE_URL')

def get_env(key: str) -> str:
    if key in os.environ:
        return os.environ[key]
    raise KeyError(f"Environment variable {key} not set.")

llm_config = {
    "model": "gpt-4p1-mini-sandbox",
    "api_type": "azure",
    "api_key": get_env("AZURE_OPENAI_API_KEY"),
    "base_url": get_env("AZURE_OPENAI_BASE_URL"),
    "api_version": "2024-12-01-preview",
    "max_tokens": 10000,
}

frontend_llm_config = {
    "model": "gpt-4p1-sandbox",
    "api_type": "azure",
    "api_key": get_env("AZURE_OPENAI_API_KEY"),
    "base_url": get_env("AZURE_OPENAI_BASE_URL"),
    "api_version": "2024-12-01-preview",
    "max_tokens": 10000,
}

# configs ----------------------------------

requirements = """
A blood volume estimator and donor eligibility tool for use by phlebotomists and clinic staff.

The system should accept the following user inputs:
- Donor Name or ID (optional)
- Age in years
- Gender (Male, Female, or Other)
- Weight (with unit: kg or lbs)
- Height (with unit: cm or inches)
- Hemoglobin (g/dL)
- Last donation date (optional)

The system should convert weight and height to metric units (kg and cm) if needed.

The system should calculate Total Blood Volume (TBV) using Nadler’s formula:
- For men: TBV = 0.3669 × height³ + 0.03219 × weight + 0.6041
- For women: TBV = 0.3561 × height³ + 0.03308 × weight + 0.1833
- For other or unknown gender: average the male and female formulas.

The system should compute the maximum allowable draw volume as 10% of the TBV, expressed in milliliters.

The system should check donor eligibility using the following criteria:
- Age must be at least 16 years
- Weight must be at least 50 kg
- Hemoglobin must be at least 12.5 g/dL
- If a last donation date is provided, it must be at least 56 days ago

The system should return structured output that includes:
- Whether the donor is eligible (✅ or ❌)
- A clear explanation if the donor is not eligible
- Estimated total blood volume (liters)
- Maximum safe draw volume (mL)
- Days since last donation (if provided)

The system should encapsulate all logic in a Python class-based module.
The system does not persist data beyond the current session.
"""

module_name = "donor_eligibility.py"
module_name_clean = "donor_eligibility"
class_name = "DonorEvaluator"

unified_work_dir = "coding"
os.makedirs(unified_work_dir, exist_ok=True)

local_code_executor = LocalCommandLineCodeExecutor(
    work_dir=unified_work_dir,
    timeout=120,
)

# Agent Definitions ----------------------------------
engineering_lead = AssistantAgent(
    name="Engineering_Lead",
    system_message=f"You are a seasoned engineering lead. Your ONLY job is to create a detailed technical design document for the `{class_name}` class based on the requirements. Start your response immediately with '# Design Document'.",
    llm_config={"config_list": [llm_config]}
)

backend_engineer = AssistantAgent(
    name="Backend_Engineer",
    system_message=f"""You are a seasoned Python engineer. Your role is to write a complete, self-contained Python module named `{module_name}` that implements the provided design.
    CRITICAL RULES: NO markdown formatting, NO explanations, ONLY pure Python code. The code must be ready to save directly to a .py file and run.""",
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,
)

test_engineer = AssistantAgent(
    name="Test_Engineer",
    system_message=f"""You are a seasoned QA engineer who writes great `unittest` tests for Python code.
    Your role is to write comprehensive unit tests for the backend module `{module_name}` based on its source code.
    CRITICAL RULES for your output:
    1. Start with `import unittest`.
    2. Include `from {module_name_clean} import {class_name}`.
    3. NO markdown formatting, NO explanations, ONLY pure Python code.
    4. End with `if __name__ == '__main__': unittest.main()`.
    """,
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,
)

frontend_engineer = AssistantAgent(
    name="Frontend_Engineer",
    system_message="""
# ROLE AND OBJECTIVE
You are an expert Frontend Engineer specializing in creating robust medical screening interfaces using the Gradio library. Your task is to write a complete, self-contained `app.py` script that uses the provided `DonorEvaluator` class to assess donor eligibility and display the results clearly.

# 1. CORE TECHNICAL INSTRUCTIONS
- The backend module `donor_eligibility.py` contains all business logic.
- You MUST import and use `DonorEvaluator`.
- The app must collect inputs such as name, age, gender, weight, height, hemoglobin, and last donation date.
- The backend returns a structured eligibility report — display all values clearly in the UI.

# 2. GRADIO REQUIREMENTS
- Use appropriate input types (e.g., Number, Radio, Date, Dropdown).
- On form submission, call backend and render:
    - ✅ or ❌ eligibility status
    - Estimated TBV in liters
    - Max draw volume in mL
    - Reason for ineligibility (if applicable)
    - Days since last donation

# 3. INTERACTION HANDLING
- Use `try/except` blocks in all backend-interfacing functions.
- Return the full set of output components in both success and error cases.
- Set `share=True` in `launch()` to allow public previewing.

# 4. OUTPUT FORMAT
Only return Python code for `app.py` — no markdown, comments, or explanation.
- Your final output must be ONLY the pure Python code for `app.py`.
- Do not include any explanations or markdown formatting like ``````.
- IMPORTANT: When launching the Gradio app, **set `share=True` in the `launch()` call** to allow public sharing.
""",
    llm_config={"config_list": [frontend_llm_config]} # Using the upgraded model
)

executor_agent = UserProxyAgent(
    name="Executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"executor": local_code_executor},
)
frontend_engineer.register_for_execution(executor_agent)

user_proxy = UserProxyAgent(
    name="Project_Manager",
    human_input_mode="NEVER",
    code_execution_config=False,
    max_consecutive_auto_reply=5,
)

# heloing functions ----------------------------------
def save_to_colab(content, filename):
    content = re.sub(r"^```(?:python)?|```$", "", content.strip(), flags=re.MULTILINE).strip()
    if not content or len(content) < 20:
        print(f"**!** Skipping save for {filename} due to empty or trivial content.")
        return None

    filepath = os.path.join(unified_work_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"**Y** Saved to {filepath}")
    return content

# workflow pipeline ----------------------------------
def run_full_stack_team():
    print("\n Starting Full-Stack Engineering Project...")

    print("\n### PHASE 1: DESIGN ###")
    design_response = user_proxy.initiate_chat(engineering_lead, message=f"Create a technical design for `{class_name}` based on: {requirements}", max_turns=2)
    design_content = save_to_colab(design_response.chat_history[-1]['content'], f"{module_name_clean}_design.md")
    if not design_content:
        print("**N** Halting: Design failed."); return

    print("\n### PHASE 2: BACKEND IMPLEMENTATION ###")
    code_response = user_proxy.initiate_chat(backend_engineer, message=f"Implement `{module_name}` based on the following design:\n\n{design_content}", max_turns=5)
    current_backend_code = save_to_colab(code_response.chat_history[-1]['content'], module_name)
    if not current_backend_code:
        current_backend_code = f"# {module_name} - Placeholder"
        save_to_colab(current_backend_code, module_name)

    print("\n### PHASE 3: TEST & REPAIR LOOP ###")
    tests_passed = False
    feedback_for_agents = "Initial test creation"

    for attempt in range(5):
        print(f"\n--- Repair Loop Attempt {attempt + 1}/5 ---")
        test_prompt = f"""
        PREVIOUS FEEDBACK:
        {feedback_for_agents}

        `{module_name}` SOURCE CODE:
        --------------------------
        {current_backend_code}
        """
        test_response = user_proxy.initiate_chat(test_engineer, message=test_prompt, max_turns=10)
        current_test_code = save_to_colab(test_response.chat_history[-1]['content'], f"test_{module_name}")
        if not current_test_code:
            feedback_for_agents = "Test code is missing. Retry!"
            continue

        print("Running unit tests...")
        execution_result = local_code_executor.execute_code_blocks([
            CodeBlock(code=f"python -m unittest test_{module_name}", language="shell")
        ])

        if execution_result.exit_code == 0:
            print("**Y** Tests Passed **Y**")
            tests_passed = True
            break

        feedback_for_agents = f"Tests failed:\n{execution_result.output}"
        if f"File \"{os.path.join(unified_work_dir, module_name)}\"" in execution_result.output:
            print("<> Updating backend...")
            repair_response = user_proxy.initiate_chat(backend_engineer, message=f"Fix the backend: {execution_result.output}", max_turns=2)
            current_backend_code = save_to_colab(repair_response.chat_history[-1]['content'], module_name)
        else:
            print("<> Updating tests...")

    if not tests_passed:
        print("**N** Final failure after retries")
        return

    print("\n### PHASE 4: FRONTEND UI GENERATION ###")
    frontend_works = False
    frontend_feedback = "Initial UI generation"

    for attempt in range(5):
        print(f"\n--- Frontend Attempt {attempt + 1}/5 ---")
        frontend_prompt = f"""
        Create `app.py` using the API of `{class_name}` below:
        --------------------------
        {current_backend_code}
        """
        frontend_response = user_proxy.initiate_chat(frontend_engineer, message=frontend_prompt, max_turns=10)
        frontend_content = save_to_colab(frontend_response.chat_history[-1]['content'], "app.py")

        validation_result = local_code_executor.execute_code_blocks([
            CodeBlock(code='python coding/app.py', language="shell")
            ])

        app_path = os.path.join(unified_work_dir, "app.py")

        if validation_result.exit_code == 0 or (os.path.exists(app_path) and os.path.getsize(app_path) > 100):
            print("**Y** Frontend validation assumed successful.")
            frontend_works = True
            break
        else:
            print("**!!!*** Frontend validation failed or app.py is missing or empty.")
            frontend_feedback = f"Execution failed when running app.py:\n{validation_result.output}"


    if not frontend_works:
        print("**N** Frontend failed after retries")
        return

    print("\n**Y** Full-Stack Engineering Project Complete! **Y**")

# Run if directly executed
if __name__ == '__main__':
    run_full_stack_team()

Streaming output truncated to the last 5000 lines.
                 hemoglobin: float = None,
                 last_donation_date: Optional[Union[str, date]] = None):
        """
        Initialize the DonorEvaluator with donor information.
        Raises ValueError for missing or invalid required parameters.
        """

    def _convert_weight_to_kg(self, weight: float, unit: str) -> float:
        """
        Converts weight to kilograms if needed.
        Raises ValueError if unit is unrecognized.
        """

    def _convert_height_to_cm(self, height: float, unit: str) -> float:
        """
        Converts height to centimeters if needed.
        Raises ValueError if unit is unrecognized.
        """

    def _calculate_tbv(self, height_cm: float, weight_kg: float, gender: str) -> float:
        """
        Calculate Total Blood Volume in liters using Nadler’s formula.
        Height is converted to meters internally for the formula.
        Gender is case-insensitive with handl

In [6]:
! python coding/test_donor_eligibility.py

.....................
----------------------------------------------------------------------
Ran 21 tests in 0.004s

OK


In [7]:
!python coding/app.py

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://9ca1b9132bfe99e698.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9ca1b9132bfe99e698.gradio.live
